# Text Study

After the initial audio study and the necessary text extraction, the next logical step will be the stduy of this type of unstructured data.

We will take a look at both the audio transcripts and the captions from the videos, in order to extract as much information as possible to predict the virality of a video.

## First libraries and variables import

In [21]:
import sys
from pathlib import Path
import os
import ast

# Get the absolute path of the folder containing the module
root_dir = Path.cwd().resolve().parent.parent

# Add the folder path to sys.path
sys.path.append(str(root_dir))

import nltk
from nltk.tokenize import word_tokenize
from InstructorEmbedding import INSTRUCTOR
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
# from mltools import regression_tools as RT

In [2]:
from text_utils import load_json, load_transcriptions, clean_text, plot_freq_dist, analyze_text, get_bag_of_words, perform_topic_modeling, classify_sentiment, count_sentiments, \
                        generate_hashtag_embeddings
from config.variables import text_path, json_file, csv_file, textcsv_file

In [3]:
text_folder = os.path.join(root_dir, text_path)
json_path = os.path.join(root_dir, json_file)

## Data recopilation and ordering

In [4]:
# Load the transcriptions of the videos
transcriptions = load_transcriptions(text_folder)

# Load the information from the JSON file
video_info = load_json(json_path)

In [5]:
# Create an object to store the combined information
video_data = {}

# Combine the information from transcriptions and the JSON file
for video_id, info in video_info.items():
    if video_id in transcriptions:
        general = info.get('general', '')  # Get the general information of the video or an empty string if not present
        text = general.get('text', '')  # Get the text of the video or an empty string if not present
        hashtags = general.get('hashtags', [])  # Get the hashtags of the video or an empty list if not present
        video_data[video_id] = {'transcription': transcriptions[video_id], 'text': text, 'hashtags': hashtags}

# Example of accessing the combined information for the first video
first_video = list(video_data.keys())[0]
print("Transcription:", video_data[first_video]['transcription'])
print("Text:", video_data[first_video]['text'])
print("Hashtags:", video_data[first_video]['hashtags'])

Transcription: 
Text: Confidence went 📈
Hashtags: []


In [6]:
# Counting the elements that are not empty strings in transcriptions
transcription_count = sum(1 for trans in transcriptions.values() if trans != '')
print("Number of non-empty transcriptions:", transcription_count)

# Counting the number of elements in video_data where the transcription is not ''
video_data_count_non_empty = sum(1 for video_id, data in video_data.items() if data['transcription'] != '')
print("Number of video_data elements with non-empty transcriptions:", video_data_count_non_empty)

# Counting the elements that are not empty strings in the video text
texts_count = sum(1 for video_id, data in video_data.items() if data['text'] != '')
print("Number of video_data elements with non-empty text:", texts_count)

# Counting the elements that are not empty lists in hashtags
hashtags_count = sum(1 for video_id, data in video_data.items() if data['hashtags'] != [])
print("Number of video_data elements with non-empty hashtags:", hashtags_count)


Number of non-empty transcriptions: 261
Number of video_data elements with non-empty transcriptions: 261
Number of video_data elements with non-empty text: 962
Number of video_data elements with non-empty hashtags: 853


As we can see, we have a transcription for 1 out of 4 videos aprox. Nevertheless, we have text and hashtags for almost all of the videos, so a text strudy could be deployed.

## Text Pre-Processing

### Text cleaning

In [7]:
# text cleaning of transcriptions and text
for video_id, data in video_data.items():
    if 'transcription' in data:
        video_data[video_id]['clean_transcription'] = clean_text(data['transcription'])
    if 'text' in data:
        video_data[video_id]['clean_text'] = clean_text(data['text'])

In [8]:
video_data[list(video_data.keys())[34]]

{'transcription': "is there the top 10 strongest one piece characters by the end of the show at number 10 we have Trafalgar law he can make anyone Immortal at the expense of his life number 9 we have useless kid he challenged kaido even though the odds are against him just like Luffy number 8 we have Sabo even though he's not at his Peak he can challenge fujitora and number 7 we have GARP the only man known to go to Toe with the former pirate king Goldie Rodger number 6 we have a kind of the author himself Oda said that if a kind of was the main character he'd find the one piece in a single year number five we have Shanks at the marineford where he just came in and said war is over and it really ended what else do you need to know about Shanks make sure like for part two",
 'text': '#Top10 Strongest #onepiece Characters by the end of the show. #anime #strongestcharacters #animeboy #luffy #zoro #animeedit #animeedits #animestiktok',
 'hashtags': ['top10',
  'onepiece',
  'anime',
  'str

As we can see, it has occured a tokenization, lemmatization and an elimination of hashtags, punctuation signs, special characters and stopwords for both text and transcription, bearing in mind the language of the possible text and transcription.

### Exploratory analysis

The next step will be the frequency distribution and the word cloud of the clean transcripts and texts, as well as the hashtags (probably the most useful one).

In [9]:
video_data

{'6907228749016714497': {'transcription': '',
  'text': 'Confidence went 📈',
  'hashtags': [],
  'clean_transcription': '',
  'clean_text': 'confidence went'},
 '6875468410612993286': {'transcription': '',
  'text': 'Quiet Zone... follow me on insta: joeysofo. Comment where you wanna see me blade next. Reply to @dwight_schnuute',
  'hashtags': [],
  'clean_transcription': '',
  'clean_text': 'quiet zone follow insta joeysofo comment wan na see blade next reply'},
 '6898699405898059010': {'transcription': '',
  'text': 'Iphone bend test🤗 #tiktok #viral #fyp #iphone #test #bend',
  'hashtags': ['tiktok', 'viral', 'fyp', 'iphone', 'test', 'bend'],
  'clean_transcription': '',
  'clean_text': 'iphone bend'},
 '6902819837345533186': {'transcription': '',
  'text': '',
  'hashtags': [],
  'clean_transcription': '',
  'clean_text': ''},
 '6905635666588192002': {'transcription': '',
  'text': '小技です👟✨#tiktok教室#tutorial',
  'hashtags': ['tiktok教室', 'tutorial'],
  'clean_transcription': '',
  'cl

In [10]:
# Collect clean transcripts, clean texts, and hashtags from video_info
clean_transcripts = []
clean_texts = []
hashtags = []
for info in video_data.values():
    if 'clean_transcription' in info:
        clean_transcripts.extend(word_tokenize(info['clean_transcription']))
    if 'clean_text' in info:
        clean_texts.extend(word_tokenize(info['clean_text']))
    if 'hashtags' in info:
        hashtags.extend(info['hashtags'])

In [11]:
# Plot frequency distribution and word cloud for clean transcripts
print("Analysis of Transcripts:")
plot_freq_dist(clean_transcripts)
analyze_text(' '.join(clean_transcripts))

Analysis of Transcripts:
<FreqDist with 2010 samples and 5247 outcomes>
Most common words: [('like', 84), ('know', 67), ('get', 46), ('going', 46), ('yeah', 46), ('want', 41), ('people', 40), ('one', 40), ('really', 35), ('way', 32)]


In [12]:
# Plot frequency distribution and word cloud for clean texts
print("Analysis of Captions:")
plot_freq_dist(clean_texts)
analyze_text(' '.join(clean_texts))

Analysis of Captions:
<FreqDist with 1953 samples and 2838 outcomes>
Most common words: [('reply', 59), ('one', 17), ('video', 15), ('day', 13), ('1', 12), ('know', 12), ('love', 11), ('jij', 10), ('de', 10), ('antwoorden', 10)]


In [13]:
# Plot frequency distribution and word cloud for hashtags
print("Analysis of Hashtags:")
plot_freq_dist(hashtags)
analyze_text(' '.join(hashtags))

Analysis of Hashtags:
<FreqDist with 2220 samples and 5330 outcomes>
Most common words: [('fyp', 417), ('foryou', 272), ('foryoupage', 174), ('fy', 116), ('fitness', 87), ('workout', 73), ('voorjou', 72), ('viral', 65), ('animeedit', 63), ('anime', 50)]


As we can see, it does not seem that a clear keyword is used in the transcripts or captions. Nevertheless, most tiktok videos has the hashtag ``fyp``, ``foryou``, ``foryoupage``, maybe in order to be recommended to users with similar interests. Furthermore, the most used ones are related with the exercise and anime.

Other techniques such as tf idf, post tagging, NER, topic modelling, etc. do not seem to have a clear usefulness for this use case. Likewise, an example is made to indicate its operation.

In [14]:
# Example usage with clean transcripts and clean texts
sentences = clean_transcripts + clean_texts
get_bag_of_words(sentences)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'let': 1664, 'place': 2176, 'getting': 1193, 'lot': 1730, 'people': 2144, 'asking': 241, 'recommending': 2382, 'use': 3074, 'sirens': 2612, 'security': 2528, 'guards': 1264, 'comments': 626, 'wanted': 3177, 'touch': 2976, 'also': 166, 'known': 1590, 'selective': 2538, 'androgenic': 189, 'receptor': 2377, 'modulators': 1898, 'held': 1329, 'next': 1990, 'generation': 1182, 'anabolic': 186, 'steroids': 2744, 'yielding': 3286, 'similar': 2603, 'results': 2430, 'without': 3245, 'side': 2598, 'effects': 908, 'true': 3019, 'starters': 2729, 'essentially': 968, 'completely': 639, 'unregulated': 3064, 'industry': 1445, 'according': 91, 'dr': 854, 'bill': 374, 'lewellen': 1672, 'likely': 1690, 'get': 1190, 'paid': 2104, 'said': 2486, 'chance': 545, 'rest': 2428, 'know': 1587, 'much': 1932, 'nobody': 2008, 'regardless': 2389, 'anybody': 207, 'might': 1859, 'tell': 2877, 'minima

In [15]:
# Example usage
topics_per_video = perform_topic_modeling(video_data)
for video_id, topics in topics_per_video.items():
    print("Topics for video", video_id)
    print(topics)

Topics for video 6907228749016714497
[(0, '1.000*"confidence went"'), (1, '1.000*"confidence went"')]
Topics for video 6875468410612993286
[(0, '1.000*"quiet zone follow insta joeysofo comment wan na see blade next reply"'), (1, '1.000*"quiet zone follow insta joeysofo comment wan na see blade next reply"')]
Topics for video 6898699405898059010
[(0, '0.143*"fyp" + 0.143*"tiktok"'), (1, '0.143*"test" + 0.143*"iphone bend"')]
Topics for video 6902819837345533186
No cleaned data available
Topics for video 6905635666588192002
[(0, '0.500*"tiktok教室" + 0.500*"tutorial"'), (1, '0.500*"tutorial" + 0.500*"tiktok教室"')]
Topics for video 6895497835681287426
[(0, '0.250*"volleyballworld" + 0.250*"great rally show love comments"'), (1, '0.250*"volleyballlove" + 0.250*"volleyball"')]
Topics for video 6895303013867539713
[(0, '1.000*"oh"'), (1, '1.000*"oh"')]
Topics for video 6884590643327290625
[(0, '1.000*"timewarpscan"'), (1, '1.000*"timewarpscan"')]
Topics for video 6906514963569888513
[(0, '0.077

It is easily observable that no large conclusions can be drawn from these exploratory and feature extraction methods. This is due to the short and incomprehensible texts of the videos and transcripts without the rest of the information.

We already knew that text was our weakest source of information, and from which we could possibly extract the least information.

### Sentiment analysis

Now, the final step of data exploration and feature extraction focuses on a basic sentiment analysis. The goal is to extract more information from the videos in order to create more accurate models, or even to consider the possibility of using models that do not rely on text, but only on hashtags and sentiments to predict their virality. Both models will be tested to observe which ones obtain more accurate results for our training and validation sets, which will be divided in the next step.

In [16]:
classify_sentiment(video_data)

In [17]:
video_data

{'6907228749016714497': {'transcription': '',
  'text': 'Confidence went 📈',
  'hashtags': [],
  'clean_transcription': '',
  'clean_text': 'confidence went',
  'text_sentiment': 'Positive',
  'transcript_sentiment': 'Unknown'},
 '6875468410612993286': {'transcription': '',
  'text': 'Quiet Zone... follow me on insta: joeysofo. Comment where you wanna see me blade next. Reply to @dwight_schnuute',
  'hashtags': [],
  'clean_transcription': '',
  'clean_text': 'quiet zone follow insta joeysofo comment wan na see blade next reply',
  'text_sentiment': 'Negative',
  'transcript_sentiment': 'Unknown'},
 '6898699405898059010': {'transcription': '',
  'text': 'Iphone bend test🤗 #tiktok #viral #fyp #iphone #test #bend',
  'hashtags': ['tiktok', 'viral', 'fyp', 'iphone', 'test', 'bend'],
  'clean_transcription': '',
  'clean_text': 'iphone bend',
  'text_sentiment': 'Neutral',
  'transcript_sentiment': 'Unknown'},
 '6902819837345533186': {'transcription': '',
  'text': '',
  'hashtags': [],
  

In [18]:
# Count the sentiments of the videos
count_sentiments(video_data)

Sentiment Counts for Text and Transcriptions (BLOB):

Sentiment  Text       Transcription
Positive   139        109       
Negative   63         45        
Neutral    798        846       
Unknown    0          0         


Sentiment Counts for Text and Transcriptions (VADER):

Sentiment  Text       Transcription
Positive   152        131       
Negative   62         50        
Neutral    786        819       
Unknown    0          0         


Sentiment Counts for Text and Transcriptions (BOTH):

Sentiment  Text       Transcription
Positive   202        136       
Negative   87         60        
Neutral    673        65        
Unknown    38         739       




As can be seen, most videos are either unlabeled or neutral, especially when looking at the transcriptions, although there appear to be more videos classified as positive sentiment than negative. Additionally, the hybrid method seems to be more effective, using both TextBlob and VADER for sentiment analysis and producing a combined response.

Finally, it's normal for the sentiments of a video's transcription and caption to differ, as captions may consist solely of hashtags or may indicate a response, among other possibilities.

### Embedding creation

The next and last step before the model creation will be the train/test split, the addition of out virality response variable, created, explained and stored in the ``project.ipynb`` and the embedding creation.

In [19]:
csv_path = os.path.join(root_dir, csv_file)

df_virality = pd.read_csv(csv_path)

data = []

for video_id, info in video_data.items():
    virality_info = df_virality[df_virality['id'] == int(video_id)]
    if not virality_info.empty:
        combined_info = {
            'video_id': video_id,
            'hashtags': info['hashtags'],
            'norm_virality': virality_info['norm_virality'].values[0]  # Add norm_virality from the DataFrame
        }
        if info['transcription']:
            combined_info['clean_transcription'] = info['clean_transcription']
            combined_info['transcript_sentiment'] = info['transcript_sentiment']
        else:
            combined_info['clean_transcription'] = ''
            combined_info['transcript_sentiment'] = ''
        if info['text']:
            combined_info['clean_text'] = info['clean_text']
            combined_info['text_sentiment'] = info['text_sentiment']
        else:
            combined_info['clean_text'] = ''
            combined_info['text_sentiment'] = ''
        data.append(combined_info) if int(video_id) in df_virality['id'].values else print(f"Video {video_id} not in virality data")
        
df = pd.DataFrame(data)

df

,video_id,hashtags,norm_virality,clean_transcription,transcript_sentiment,clean_text,text_sentiment
0,6907228749016714497,[],0.000177,,,confidence went,Positive
1,6875468410612993286,[],0.003337,,,quiet zone follow insta joeysofo comment wan n...,Negative
2,6898699405898059010,"[tiktok, viral, fyp, iphone, test, bend]",0.060929,,,iphone bend,Neutral
3,6902819837345533186,[],0.000377,,,,
4,6905635666588192002,"[tiktok教室, tutorial]",0.000457,,,,Neutral
...,...,...,...,...,...,...,...
995,6877191692341054721,"[foryou, foryoupage]",0.000513,hi please scroll seen video means small creato...,Negative,,Neutral
996,6908069845825359109,"[gttfg, gotothegym, swolefam, nutrition, diet,...",0.000320,fuck pumpkin spice stop shoveling garbage mout...,Negative,stop eating,Negative
997,6883484287434378497,"[fy, foryoupage, foryou]",0.001790,,,,Neutral
998,6898721943978036481,"[fyp, foryou, curls]",0.000286,imagine long beautiful hair one day wake think...,Positive,regretss,Neutral


Prior to the model creation, an embedding model is used to transform the clean text, transcript and hashtags. It could also be investigated with different embedding models, but it is not the objective of this work, so a HuggingFace model with good general results is chosen.

In [33]:
# # First execution may be long due to the model download

# model = INSTRUCTOR('hkunlp/instructor-xl')
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [34]:
# # Text embedding creation 
# embeddings = model.encode(sentences=df['clean_text'].values, show_progress_bar=True, device=device, normalize_embeddings=True)

In [35]:
# # Text embedding storage
# df['text_embedding'] = embeddings.tolist()

In [36]:
# # Transcription embedding creation & storage
# embeddings = model.encode(sentences=df['clean_transcription'].values, show_progress_bar=True, device=device, normalize_embeddings=True)
# df['transcription_embedding'] = embeddings.tolist()

In [37]:
# # Tokenize hashtags into lists of words
# tokenized_hashtags = [hashtag.split() for hashtag in hashtags]

In [38]:
# # Train Word2Vec model for hashtags
# hashtag_model = Word2Vec(sentences=tokenized_hashtags, vector_size=100, window=5, min_count=1, workers=8, seed=42)

# # Apply the function to create embeddings for each row in the DataFrame
# df['hashtag_embeddings'] = df['hashtags'].apply(lambda hashtags: generate_hashtag_embeddings(hashtags, hashtag_model))

In [39]:
# df

In [40]:
# df.to_csv(os.path.join(root_dir, textcsv_file, 'embeddings.csv'), index=False)

In [41]:
df = pd.read_csv(os.path.join(root_dir, textcsv_file, 'embeddings.csv'))

In [42]:
df

,video_id,hashtags,norm_virality,clean_transcription,transcript_sentiment,clean_text,text_sentiment,text_embedding,transcription_embedding,hashtag_embeddings
0,6907228749016714497,[],0.000177,NaN,NaN,confidence went,Positive,"[0.016430748626589775, -0.0017317278543487191,...","[0.04192333295941353, -0.007496909704059362, -...",[]
1,6875468410612993286,[],0.003337,NaN,NaN,quiet zone follow insta joeysofo comment wan n...,Negative,"[-0.0047773574478924274, -0.006476661190390587...","[0.04192333295941353, -0.007496909704059362, -...",[]
2,6898699405898059010,"['tiktok', 'viral', 'fyp', 'iphone', 'test', '...",0.060929,NaN,NaN,iphone bend,Neutral,"[-0.025292418897151947, 0.010925445705652237, ...","[0.04192333295941353, -0.007496909704059362, -...","[array([-2.6800192e-03, -9.8498594e-03, -9.216..."
3,6902819837345533186,[],0.000377,NaN,NaN,NaN,NaN,"[0.04192333295941353, -0.007496909704059362, -...","[0.04192333295941353, -0.007496909704059362, -...",[]
4,6905635666588192002,"['tiktok教室', 'tutorial']",0.000457,NaN,NaN,NaN,Neutral,"[0.04192333295941353, -0.007496909704059362, -...","[0.04192333295941353, -0.007496909704059362, -...","[array([ 0.00561919, -0.00724378, -0.00390926,..."
...,...,...,...,...,...,...,...,...,...,...
995,6877191692341054721,"['foryou', 'foryoupage']",0.000513,hi please scroll seen video means small creato...,Negative,NaN,Neutral,"[0.04192333295941353, -0.007496909704059362, -...","[0.05268768221139908, -0.007747612427920103, -...","[array([ 6.6547347e-03, -6.0018362e-03, 6.097..."
996,6908069845825359109,"['gttfg', 'gotothegym', 'swolefam', 'nutrition...",0.000320,fuck pumpkin spice stop shoveling garbage mout...,Negative,stop eating,Negative,"[0.016533860936760902, -0.023285886272788048, ...","[0.02685837633907795, 0.05185440927743912, 0.0...","[array([ 0.00014931, -0.00081907, -0.00045761,..."
997,6883484287434378497,"['fy', 'foryoupage', 'foryou']",0.001790,NaN,NaN,NaN,Neutral,"[0.04192333295941353, -0.007496909704059362, -...","[0.04192333295941353, -0.007496909704059362, -...","[array([-0.00272644, 0.00652584, -0.00980168,..."
998,6898721943978036481,"['fyp', 'foryou', 'curls']",0.000286,imagine long beautiful hair one day wake think...,Positive,regretss,Neutral,"[0.016412628814578056, -0.05790542811155319, 0...","[0.04319428279995918, -0.04913891479372978, 0....","[array([-8.2149813e-03, 5.4791202e-03, 3.091..."


### Train/Test split

It will be used **$80\%$** of the data for training, **$10\%$** for validation and **$10\%$** for testing the correctness and lack of overfitting of the final model.

In [43]:
train_df, Test_df = train_test_split(df, test_size=0.2, random_state=42)

validation_df, test_df = train_test_split(Test_df, test_size=0.5, random_state=42)

In [44]:
test_df

,video_id,hashtags,norm_virality,clean_transcription,transcript_sentiment,clean_text,text_sentiment,text_embedding,transcription_embedding,hashtag_embeddings
436,6898628429298568450,"['excavator', 'jurassicpark']",0.000268,NaN,NaN,NaN,Neutral,"[0.04192333295941353, -0.007496909704059362, -...","[0.04192333295941353, -0.007496909704059362, -...","[array([ 0.00674202, -0.00799041, 0.00640684,..."
899,6903974884154608897,"['foryou', 'fy']",0.000127,NaN,NaN,uuuuhhhh,Neutral,"[-0.01943052001297474, -0.028604578226804733, ...","[0.04192333295941353, -0.007496909704059362, -...","[array([ 6.6547347e-03, -6.0018362e-03, 6.097..."
346,6894047232274812165,"['fyp', 'foryou', 'viral', 'makeupartist', 'ma...",0.000302,NaN,NaN,okayyy transition tho also find first help,Positive,"[-0.007570543326437473, 0.001123668160289526, ...","[0.04192333295941353, -0.007496909704059362, -...","[array([-8.2149813e-03, 5.4791202e-03, 3.091..."
60,6896962911969758466,"['fyp', 'foryou', 'fy', 'freestylefootball', '...",0.000176,ok google,Positive,hard work,Negative,"[0.031882815062999725, 0.00465204706415534, -0...","[0.024793338030576706, 0.022074377164244652, 0...","[array([-8.2149813e-03, 5.4791202e-03, 3.091..."
867,6886298413885918465,[],0.000177,NaN,NaN,NaN,NaN,"[0.04192333295941353, -0.007496909704059362, -...","[0.04192333295941353, -0.007496909704059362, -...",[]
...,...,...,...,...,...,...,...,...,...,...
213,6900185373540961537,"['world', 'facts', 'list', 'gaytiktok', 'lovei...",0.000122,NaN,NaN,naast levenslange straf landen doodstraf uitvo...,Neutral,"[0.010551031678915024, 0.0029549868777394295, ...","[0.04192333295941353, -0.007496909704059362, -...","[array([-9.3629099e-03, 6.1428929e-03, -7.818..."
884,6885791166248340737,"['duet', 'duet', 'mensfitness', 'menshealth', ...",0.000132,seems tik tok lot people believe experts withi...,Positive,first duet young chap,Positive,"[0.018415972590446472, -0.050034280866384506, ...","[0.0614435113966465, -0.024465279653668404, -0...","[array([ 5.1929462e-03, -1.9728900e-03, 3.996..."
626,6895317957522541825,"['fyp', 'viral', 'comedy']",0.001672,NaN,NaN,embarrassing moment ever happened chersmarijke...,Negative,"[0.03151969239115715, 0.00880822166800499, -0....","[0.04192333295941353, -0.007496909704059362, -...","[array([-8.2149813e-03, 5.4791202e-03, 3.091..."
986,6902823504383659266,[],0.000323,NaN,NaN,vandaag alle muren ons huis geverfd,Neutral,"[0.013059204444289207, -0.005901109892874956, ...","[0.04192333295941353, -0.007496909704059362, -...",[]


## Text Model Creation

Before continuing with the creation of the models, it is necessary to explain the problem that concerns us. 

Unlike traditional text NLP models, for each observation we have more than one variable with associated text. Therefore, 3 different techniques are available:

 + Embedding transformation of each variable and use them separately.

 + Mixing all embeddings generated by caption, transcription and hashtags.

 + Creation of traditional models using the extracted characteristics (text sentiment, transcription sentiment and hashtags) as qualitative variables.

The first and second cases are more related to text analysis, so they will be the most developed. However, a couple of models will also be made according to the third idea to observe their qualities, using models that are usually great options for prediction such as MLP and gradient boosting (universal approximators).

### First approach

Due to the fact that every variable is a vector, we cannot use a model whose variables are vectors. Therefore, we are going to concatenate all embeddings into a single vector. It could also be deployed a model for each variable and a fourth model (a MLP, for instance) to learn how to mix them properly and apply correct weights in order to minimize the loss, but it is a method that needs more resources than the current ones, so it is not going to be tested.

In [61]:
# Defining features and target variables
features = ["text_embedding", "transcription_embedding", "hashtag_embeddings"]
target = "norm_virality"

# Training set
x_train = train_df[features]
y_train = train_df[target]

# Validation set
x_valid = validation_df[features]
y_valid = validation_df[target]

# Test set
x_test = test_df[features]
y_test = test_df[target]

#### ML models

In [64]:
features

['text_embedding', 'transcription_embedding', 'hashtag_embeddings']

In [59]:
INPUTS_LR_NUM = features
INPUTS_LR_CAT = []
INPUTS_LR = INPUTS_LR_NUM + INPUTS_LR_CAT

# Convert string representations of lists to actual lists
for feature in INPUTS_LR_NUM:
    if feature == 'hashtag_embeddings':
        for hashtag in 

    else:
        x_train[feature] = x_train[feature].apply(ast.literal_eval)

C:\Users\juanl\AppData\Local\Temp\ipykernel_54712\2871709438.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[feature] = x_train[feature].apply(ast.literal_eval)


ValueError: malformed node or string on line 1: <ast.Call object at 0x000001A883D0BF70>

In [49]:
## Inputs of the model. Change accordingly to perform variable selection
INPUTS_LR_NUM = features
INPUTS_LR_CAT = []
INPUTS_LR = INPUTS_LR_NUM + INPUTS_LR_CAT

# Prepare the numeric variables by imputing by scaling
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# Prepare the categorical variables by encoding the categories
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore',drop='first'))])

# Create a preprocessor to perform the steps defined above
preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, INPUTS_LR_NUM),
        ('cat', categorical_transformer, INPUTS_LR_CAT)
        ])

pipe = Pipeline(steps=[('Prep',preprocessor), # Preprocess the variables when training the model 
                        ('LinReg', LinearRegression())]) 


# We use Grid Search Cross Validation to find the best parameter for the model in the grid defined 
nFolds = 10
param = {}
LR_fit = GridSearchCV(estimator=pipe, # Structure of the model to use
                        param_grid=param, # Defined grid to search in
                        n_jobs=-1, # Number of cores to use (parallelize)
                        cv=nFolds) # Number of Folds 
LR_fit.fit(x_train[INPUTS_LR], y_train) # Search in grid

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\juanl\Documents\Master\2o cuatri\ADNE\Trabajo\Tik Tok\tiktok\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\juanl\Documents\Master\2o cuatri\ADNE\Trabajo\Tik Tok\tiktok\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\juanl\Documents\Master\2o cuatri\ADNE\Trabajo\Tik Tok\tiktok\Lib\site-packages\sklearn\pipeline.py", line 471, in fit
    Xt = self._fit(X, y, routed_params)
         ^^^^^^^^^^^^^